# Experiment v01 -
## Classic ML vs Hetero-GNN-Benchmark - POC

* Data:
    * Synthetic Data will be generated about 1000 people, and 10000 diagnoses related to them
    * People will get assigned labels depending on whether they had a specific diagnosis or not

* Split:
    * Data will be split 60:20:20 int train, val and test splits

* Classic ML vs GNN:
    * GradientBoosting Model will be trained on peoples properties to predict the label
        * Features: age, gender, diagnoses (one-hot encoded)
    * HGNN:
        * Features people: age
        * Features diagnoses: age and icd code (one-hot encoded)

In [ ]:
import os
import torch
import neo4j
import time
import string
import datetime
import testkasse
import sqlalchemy
import numpy as np
import pandas as pd
import networkx as nx
from sklearn import preprocessing
from torch.nn import Linear
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
import torch_geometric.transforms as T
from torch_geometric.data import HeteroData
from torch_geometric.transforms import ToUndirected
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from dotenv import load_dotenv
from torch_geometric.data import Dataset, Batch
from torch.nn import Linear, ReLU, Sequential, LeakyReLU
from torch_geometric.data import Dataset
from torch_geometric.loader import HGTLoader, NeighborLoader, DataLoader
from torch_geometric.nn import (
    SAGEConv,
    to_hetero,
    to_hetero_with_bases,
    global_max_pool,
    MeanAggregation,
    MinAggregation,
    GraphConv,
    MaxAggregation,
)
from torch import Tensor
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score
from torch_geometric.nn import BatchNorm, GraphConv
from torch_geometric.nn import MultiAggregation

load_dotenv()
from yfiles_jupyter_graphs import GraphWidget
from utils_neo4j import *
from utils_middleware_v1 import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
session = driver.session()
result = session.run("MATCH (v)-[r]->(d) RETURN v,r,d LIMIT 20")
w = GraphWidget(graph=result.graph())
w.show()

## Generate Samples - Versicherte & Diagnosen

In [ ]:
NUM_SAMPLES = 10_000

backend = sqlalchemy.create_engine("sqlite:///test.db")

testkasse.generate_all_tables(anzahl_versicherte=NUM_SAMPLES, conn=backend)

## Create Splits

In [ ]:
stammdaten = pd.read_sql_table("stammdaten", con=backend)
icd_data = pd.read_sql_table("kh", con=backend)

train, validate, test = np.split(
    stammdaten.sample(frac=1), [int(0.6 * len(stammdaten)), int(0.8 * len(stammdaten))]
)
stammdaten.loc[train.index, "mode"] = "train"
stammdaten.loc[validate.index, "mode"] = "validate"
stammdaten.loc[test.index, "mode"] = "test"

## Feature Engineering to make data compatible with Classic ML Models

In [ ]:
# Create Short ICD Codes
icd_data["icd_short"] = icd_data["ICD"].str[:1]

scaler2 = StandardScaler()

icd_data = pd.read_sql_table("kh", con=backend)

# Get Geschlecht und Alter
le = preprocessing.LabelEncoder()
le.fit(stammdaten[stammdaten["mode"] != "test"].Geschlecht)
stammdaten["geschlecht_encoded"] = le.transform(stammdaten.Geschlecht)

stammdaten["now"] = pd.to_datetime(datetime.datetime.now())
stammdaten["alter"] = (stammdaten[["Todestag", "now"]].min(axis=1) - stammdaten.Geburtstag).dt.days
stammdaten["alter"] = scaler2.fit_transform(np.expand_dims(pd.to_numeric(stammdaten["alter"], downcast="float"), 1))

# Get icd codes
scaler = MaxAbsScaler()
icd_data = icd_data.merge(stammdaten[["KVNR", "Geburtstag", "mode"]], on="KVNR", how="left")
icd_data["icd_age"] = (icd_data["kh_von"] - icd_data.Geburtstag).dt.days
scaler.fit(icd_data[icd_data["mode"] != "test"][["icd_age"]])
icd_data["icd_age"] = scaler.transform(icd_data[["icd_age"]])
icd_data["icd_short"] = icd_data["ICD"].str[:1].apply(str.lower)
icd_agg = icd_data.groupby("KVNR")["icd_short"].apply(" ".join)

merged = stammdaten.merge(icd_agg, how="left", on="KVNR")


def analyzer_custom(doc):
    return doc.split()


vectorizer = CountVectorizer(vocabulary=list(string.ascii_lowercase), analyzer=analyzer_custom)
merged["icd_short"] = merged["icd_short"].fillna("a")
data = vectorizer.transform(merged["icd_short"]).todense()
merged.loc[:, "icd_vec"] = data.tolist()
X = np.hstack([merged[["geschlecht_encoded", "alter"]].values, np.vstack(merged["icd_vec"].values)])

In [ ]:
# TODO: Add Min, Max, Mean and Sum of ages of each ICD to the featurelist of the GradientBoosting Method to provide it with more relevant info
# def prop_to_vec(val):
#    x = np.zeros((1, 26))
#    x[0, ord(val) - 97] = 1
#    return x

# icd_data["inds"] = icd_data["ICD"].str[:1].apply(str.lower).apply(prop_to_vec)
# icd_data["inds"] = icd_data["inds"] * icd_data.icd_age
# icd_data["inds"] = np.zeros((10000, 26)).tolist()
# icd_data["min_vecs"] = icd_data[["KVNR", "inds"]].groupby("KVNR").max()
# icd_data

In [ ]:
# icd_data

## Create Labels

In [ ]:
# Create new Label: Was diagnosed very early with specific Diagnosis
icd_data["label"] = (icd_data["icd_short"] <= "d") & (icd_data["icd_age"] < 0)
stammdaten = stammdaten.merge(icd_data[["KVNR", "label"]].groupby("KVNR").max(), on="KVNR", how="left")
stammdaten["label"] = stammdaten["label"].fillna(False)

In [ ]:
icd_data.head(2)

In [ ]:
stammdaten.head(2)

## Classic ML Approach

In [ ]:
# Now train classic ML model (XGBoost)
train_inds = stammdaten[stammdaten["mode"] == "train"].index.tolist()
val_inds = stammdaten[stammdaten["mode"] == "validate"].index.tolist()
test_inds = stammdaten[stammdaten["mode"] == "test"].index.tolist()

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(
    X[train_inds], stammdaten.loc[train_inds, ["label"]].astype(int).values.ravel()
)

print(
    "Accuracy Test Daten, GradientBoosting:",
    clf.score(X[val_inds], stammdaten.loc[val_inds, ["label"]].astype(int).values.ravel()),
)

print("Features: Geschlecht, Alter, ICD-One-Hot-encoded")

In [ ]:
feats = ["feature_" + str(i) for i in range(X.shape[1])]

In [ ]:
df_train = pd.concat(
    [
        pd.DataFrame(data=X[train_inds].tolist(), columns=feats),
        pd.DataFrame({"label": stammdaten.loc[train_inds, ["label"]].astype(int).values.ravel()}),
    ],
    axis=1,
)
df_test = pd.concat(
    [
        pd.DataFrame(data=X[test_inds].tolist(), columns=feats),
        pd.DataFrame({"label": stammdaten.loc[test_inds, ["label"]].astype(int).values.ravel()}),
    ],
    axis=1,
)

In [ ]:
df_train

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset(df_train)
test_data = TabularDataset(df_test)

predictor = TabularPredictor(label="label").fit(train_data=train_data)

In [ ]:
# predictor = TabularPredictor.load("AutogluonModels/ag-20230831_124650/")
predictions = predictor.predict(test_data)

import sklearn

sklearn.metrics.accuracy_score(predictions, test_data.label.tolist())

## Add Data to neo4j

In [ ]:
conn.query("DROP CONSTRAINT DIAGNOSE")
conn.query("CREATE CONSTRAINT versicherter IF NOT EXISTS FOR (v:versicherter) REQUIRE v.KVNR IS UNIQUE")
conn.query("MATCH (n) DETACH DELETE n")

In [ ]:
add_diagnosen_v1(icd_data[["ICD", "icd_age", "icd_short"]])

In [ ]:
add_versicherte_v1(stammdaten)

In [ ]:
conn.query("DROP CONSTRAINT DIAGNOSE")
conn.query("CREATE CONSTRAINT versicherter IF NOT EXISTS FOR (v:versicherter) REQUIRE v.KVNR IS UNIQUE")
conn.query("MATCH (n) DETACH DELETE n")

add_diagnosen_v1(icd_data[["ICD", "icd_age", "icd_short"]])
add_versicherte_v1(stammdaten)
add_relations_v1(icd_data.merge(stammdaten, on="KVNR"))

# conn.query("MATCH (v: versicherter {KVNR: '" + icd_data.iloc[1].KVNR + "'})-[:DIAGNOSTIZIERT]-(d) RETURN v, d")

## Create Dataloader-Helper-Function

In [ ]:
def get_single_graph(idx=0):
    versicherter_query = (
        """
    MATCH (v:versicherter {id: '"""
        + stammdaten.iloc[idx].KVNR
        + """'})-[:DIAGNOSTIZIERT]->(d:diagnose)
    WITH v, collect(d.name) AS diagnose_list
    RETURN v.id AS versichertenId, v.alter AS alter, v.KVNR AS KVNR, v.label AS label
    """
    )

    versicherter_x_, versicherter_mapping_ = load_node(
        versicherter_query,
        index_col="versichertenId",
        encoders={
            "alter": ScalarIdentityEncoder(torch.float),
        },
    )

    diagnose_query = (
        """
    MATCH (v:versicherter {id: '"""
        + stammdaten.iloc[idx].KVNR
        + """'})-[:DIAGNOSTIZIERT]->(d:diagnose)
    WITH d
    RETURN d.name AS diagnoseId, d.name AS name, d.icd_age AS icd_age, d.icd_short AS icd_short
    """
    )

    diagnose_x_, diagnose_mapping_ = load_node(
        diagnose_query,
        index_col="diagnoseId",
        encoders={"icd_short": IcdEncoder(), "icd_age": ScalarIdentityEncoder()},
    )

    edge_query = (
        """
    MATCH (v:versicherter {id: '"""
        + stammdaten.iloc[idx].KVNR
        + """'})-[r:DIAGNOSTIZIERT]->(d:diagnose) 
    RETURN v.id AS vId, d.name AS dId
    """
    )

    edge_index_, edge_label_ = load_edge(
        edge_query,
        src_index_col="vId",
        src_mapping=versicherter_mapping_,
        dst_index_col="dId",
        dst_mapping=diagnose_mapping_,
    )

    data_ = HeteroData()
    data_["versicherter"].x = versicherter_x_
    data_["versicherter"].label = torch.from_numpy(np.array(int(stammdaten.iloc[idx].label)))
    data_["diagnose"].x = diagnose_x_.float()
    data_["versicherter", "hat", "diagnose"].edge_index = edge_index_
    data_ = ToUndirected()(data_)
    data_.to(device, non_blocking=True)
    return data_

## Declare Dataset

In [ ]:
class MyOwnDataset(Dataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None, mode="train"):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.mode = mode
        if mode == "train":
            self.inds = train.index.tolist()
        elif mode == "val":
            self.inds = validate.index.tolist()
        elif mode == "test":
            self.inds = test.index.tolist()
        # print(self.inds)

    @property
    def raw_file_names(self):
        return [
            "some_file_1",
        ]

    @property
    def processed_file_names(self):
        return [
            "data_1.pt",
        ]

    def download(self):
        pass

    def process(self):
        pass

    def len(self):
        return len(self.inds)

    def get(self, idx):
        data = get_single_graph(self.inds[idx])
        return data

## Declare DataLoaders

In [ ]:
dset_train = MyOwnDataset(root="", mode="train")
dset_val = MyOwnDataset(root="", mode="val")
dset_test = MyOwnDataset(root="", mode="test")

In [ ]:
"""
from collections import OrderedDict

batch_size = 32
hidden_size = 4
num_classes = 2
learn_rate = 0.01
aggr="max"


class GraphLevelGNN(torch.nn.Module):
    def __init__(self, config={
            "hidden_size": hidden_size,
            "activation": F.leaky_relu,
            "batchnorm": BatchNorm,
            "enable_dropout": False,
            "graphlayer": GraphConv,
            "num_graphlayers": 2}):
        super().__init__()
        self.config = config

        self.body = F.nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ]))
        #self.conv1 = GraphConv(-1, hidden_size)
        #self.bn1 = BatchNorm(hidden_size)
        self.pool = MultiAggregation(
            aggrs=['mean', 'min', 'max'],
            mode="cat"
            )
        self.lin = Linear(hidden_size*3, num_classes)

    def forward(self, x: Tensor, edge_index: Tensor, batch: Tensor) -> Tensor:
        x = self.conv1(x, edge_index)
        x = F.leaky_relu(x)
        x = self.bn1(x)
        x = self.conv2(x, edge_index)
        x = self.pool(x, batch)
        x = self.lin(x)
        return x

g1 = get_single_graph(647)
g2 = get_single_graph(1)
btch = Batch.from_data_list([g1, g2])

metadata = g1.metadata()

model = GraphLevelGNN()
model = to_hetero(model, metadata, aggr=aggr, debug=True)
optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)

out = model(btch.x_dict, btch.edge_index_dict, btch.batch_dict)
"""

In [ ]:
g1 = dset_train.get(0)
g1

In [ ]:
import torch_geometric


g3 = T.AddSelfLoops()(g1)
g3

In [ ]:
# OGB_MAG(root='./data', preprocess='metapath2vec', transform=T.ToUndirected())

In [ ]:
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv, GATConv, Linear, DeepGCNLayer

batch_size = 32
hidden_size = 27
num_classes = 2
learn_rate = 0.01
aggr = "max"


import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.nn import SAGEConv, to_hetero


# dataset = OGB_MAG(root='./data', preprocess='metapath2vec', transform=T.ToUndirected())
data = dset_train[0]


class GraphLevelGNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.convX = GraphConv(-1, 32)
        self.pool = MultiAggregation(aggrs=["mean", "min", "max"], mode="cat")
        self.lin = Linear(hidden_size * 3, num_classes)

    def forward(self, x: Tensor, edge_index: Tensor, batch: Tensor) -> Tensor:
        x = self.convX(x, edge_index)
        x = F.leaky_relu(x)
        x = self.pool(x, batch)
        x = self.lin(x)
        return x


from torch.utils.checkpoint import checkpoint


class GraphLevelGNNRes(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GraphConv(-1, 27)
        self.conv2 = GraphConv(-1, 27)
        self.conv3 = GraphConv(-1, 27)
        self.batchnorm1 = BatchNorm(27)
        self.pool = MultiAggregation(aggrs=["mean", "min", "max"], mode="cat")
        self.lin = Linear(hidden_size * 3, num_classes)

    def forward(self, x: Tensor, edge_index: Tensor, batch: Tensor) -> Tensor:
        # h = checkpoint(self.conv1, x)
        h1 = self.conv1(x, edge_index)
        h1 = self.batchnorm1(h1)
        h1 = F.leaky_relu(h1)
        h1 = x + h1
        h1 = F.dropout(h1, p=0.1, training=self.training)

        h2 = self.conv2(h1, edge_index)
        h2 = self.batchnorm1(h2)
        h2 = F.leaky_relu(h2)
        h2 = h1 + h2
        h2 = F.dropout(h2, p=0.1, training=self.training)

        h3 = self.conv3(h2, edge_index)
        h3 = self.batchnorm1(h3)
        h3 = F.leaky_relu(h3)
        h3 = h2 + h3
        h3 = F.dropout(h3, p=0.1, training=self.training)

        h3 = self.pool(h3, batch)
        h3 = self.lin(h3)
        return h3


g1 = get_single_graph(647)
g2 = get_single_graph(1)

btch = Batch.from_data_list([g1, g2])

metadata = g1.metadata()

model = GraphLevelGNNRes()
model = to_hetero(model, metadata, aggr=aggr, debug=False)
optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)

out = model(btch.x_dict, btch.edge_index_dict, btch.batch_dict)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm


writer = SummaryWriter("logs/eperiment-01-single_graphconv")


train_loader = DataLoader(dset_train, batch_size=32, shuffle=True, num_workers=12)
val_loader = DataLoader(dset_val, batch_size=16, shuffle=False, num_workers=12)
test_loader = DataLoader(dset_test, batch_size=16, shuffle=False, num_workers=12)

# writer.add_hparams({
#    "train_batch_size": batch_size
# })


def train(epoch=0):
    model.train()
    preds = []
    ytrue = []
    all_loss = 0.0
    for btch in tqdm(train_loader):
        optimizer.zero_grad()
        out = model(btch.x_dict, btch.edge_index_dict, btch.batch_dict)
        loss = F.cross_entropy(out, btch["versicherter"].label)
        loss.backward()
        optimizer.step()
        preds.extend(torch.argmax(torch.softmax(out, dim=-1), dim=-1).detach().tolist())
        ytrue.extend(btch["versicherter"].label.tolist())
        all_loss += loss.item()
    acc = accuracy_score(ytrue, preds)
    print("Train Loss:", all_loss)
    print("Train Acc:", acc)
    writer.add_scalar("Loss/train", all_loss, epoch)
    writer.add_scalar("Acc/train", acc, epoch)
    all_loss = 0.0
    preds = []
    ytrue = []


@torch.no_grad()
def test(epoch=0):
    model.eval()
    preds = []
    ytrue = []
    all_loss = 0.0
    for btch in tqdm(test_loader):
        out = model(btch.x_dict, btch.edge_index_dict, btch.batch_dict)
        loss = F.cross_entropy(out, btch["versicherter"].label)
        preds.extend(torch.argmax(torch.softmax(out, dim=-1), dim=-1).detach().tolist())
        ytrue.extend(btch["versicherter"].label.tolist())
        all_loss += loss.item()
    acc = accuracy_score(ytrue, preds)
    print("Test Loss:", all_loss)
    print("Test Acc:", acc)
    writer.add_scalar("Loss/test", all_loss, epoch)
    writer.add_scalar("Acc/test", acc, epoch)
    all_loss = 0.0
    preds = []
    ytrue = []


for i in tqdm(range(10)):
    train(i)
    test(i)

In [ ]:
%load_ext tensorboard

In [ ]:
#%tensorboard --logdir logs